In [9]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install mysql-replication

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pymysqlreplication import BinLogStreamReader
from pymysqlreplication import row_event
import configparser
import pymysqlreplication

In [12]:
parser = configparser.ConfigParser()
parser.read("pipeline.ini")
hostname = parser.get("mysql_config", "hostname")
port = parser.get("mysql_config", "port")

In [13]:
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")
mysql_settings = {
    "host": hostname,
    "port": int(port),
    "user": username,
    "passwd": password
}

In [14]:
b_stream = BinLogStreamReader(connection_settings = mysql_settings,server_id=100,only_events=[row_event.DeleteRowsEvent,row_event.WriteRowsEvent,row_event.UpdateRowsEvent])

In [16]:
order_events = []

In [17]:
for binlogevent in b_stream:
    for row in binlogevent.rows:
        if binlogevent.table == 'orders':
            event = {}
            if isinstance(binlogevent,row_event.DeleteRowsEvent):
                event["action"] = "delete"
                event.update(row["values"].items())
            elif isinstance(binlogevent,row_event.UpdateRowsEvent):
                event["action"] = "update"
                event.update(row["after_values"].items())
            elif isinstance(binlogevent,row_event.WriteRowsEvent):
                event["action"] = "insert"
                event.update(row["values"].items())
                order_events.append(event)

                       Setting The Variable Value BINLOG_ROW_METADATA = FULL, BINLOG_ROW_IMAGE = FULL.
                       By Applying this, provide properly mapped column information on UPDATE,DELETE,INSERT.
                        


In [18]:
b_stream.close()